![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Building Multi-Tool Intelligence: Semantic Tool Selection

## Welcome to Section 4: Semantic Tool Selection

In Section 3, you enhanced your agent with sophisticated memory. Now you'll add multiple specialized tools and intelligent routing that can understand user intent and select the right tool for each query.

Your agent will evolve from a simple course recommender to a comprehensive academic advisor with multiple capabilities.

## Learning Objectives

By the end of this notebook, you will:
1. Add multiple specialized tools to your memory-enhanced agent
2. Implement semantic tool selection using embeddings
3. Build intent classification with confidence scoring
4. Create memory-aware tool routing
5. Test complex multi-tool scenarios

## The Tool Selection Problem

As your agent gains more capabilities, tool selection becomes critical:

### Cross-Reference: Tool Selection Challenges

This builds on concepts from the original tool notebooks:
- `section-2-system-context/02_defining_tools.ipynb` - What tools are and why they're essential
- `section-2-system-context/03_tool_selection_strategies.ipynb` - Common tool selection failures

**With Few Tools (Section 2):**
```
User: "What courses should I take?"
Agent: Uses course search tool ✅
```

**With Many Tools (Section 4):**
```
User: "What courses should I take?"
Available tools: search_courses, get_recommendations, check_prerequisites, 
                 check_schedule, enroll_student, track_progress...
Agent: Which tool? 🤔
```

**Solution: Semantic Tool Selection**
- Understand user intent using embeddings
- Match queries to tool capabilities semantically
- Use memory to inform tool selection
- Provide confidence scoring and fallbacks

## Step 1: Load Your Memory-Enhanced Agent

First, let's load the memory-enhanced agent you built in Section 3 as our foundation.

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify required environment variables are set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY not found. Please create a .env file with your OpenAI API key. "
        "Get your key from: https://platform.openai.com/api-keys"
    )

print("✅ Environment variables loaded")
print(f"   REDIS_URL: {os.getenv('REDIS_URL', 'redis://localhost:6379')}")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")

# Import components from previous sections
import sys
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
import json

# Add reference agent to path
sys.path.append('../../reference-agent')

from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel, 
    CourseFormat, Semester
)
from redis_context_course.course_manager import CourseManager
from redis_context_course.tools import create_course_tools
from redis_context_course.semantic_tool_selector import SemanticToolSelector

# Import tool components
from langchain_core.tools import BaseTool, tool
from langchain_openai import OpenAIEmbeddings

print("Foundation components loaded")

## Step 2: Create Specialized Tools

Let's create multiple specialized tools that your agent can use for different academic advisor tasks.

In [ ]:
# Define specialized tools for academic advising
class AcademicAdvisorTools:
    """Collection of specialized tools for academic advising"""
    
    def __init__(self, course_manager: CourseManager):
        self.course_manager = course_manager
        self.tools = self._create_tools()
    
    def _create_tools(self) -> List[Dict[str, Any]]:
        """Create all specialized tools"""
        return [
            {
                "name": "search_courses",
                "description": "Search for courses by topic, level, or keywords. Use when students want to explore available courses.",
                "function": self.search_courses,
                "examples": [
                    "What machine learning courses are available?",
                    "Show me beginner programming courses",
                    "Find courses about data science"
                ],
                "keywords": ["search", "find", "show", "available", "courses", "list"]
            },
            {
                "name": "get_recommendations",
                "description": "Get personalized course recommendations based on student profile and goals. Use when students ask what they should take.",
                "function": self.get_recommendations,
                "examples": [
                    "What courses should I take next?",
                    "Recommend courses for my career goals",
                    "What's the best learning path for me?"
                ],
                "keywords": ["recommend", "suggest", "should", "best", "next", "path"]
            },
            {
                "name": "check_prerequisites",
                "description": "Check if a student meets prerequisites for specific courses. Use when students ask about course requirements.",
                "function": self.check_prerequisites,
                "examples": [
                    "Can I take RU301?",
                    "Do I meet the requirements for advanced courses?",
                    "What prerequisites do I need?"
                ],
                "keywords": ["prerequisites", "requirements", "can I take", "eligible", "qualify"]
            },
            {
                "name": "check_schedule",
                "description": "Check course schedules and availability. Use when students ask about timing or scheduling.",
                "function": self.check_schedule,
                "examples": [
                    "When is RU201 offered?",
                    "What's the schedule for machine learning courses?",
                    "Are there evening classes available?"
                ],
                "keywords": ["schedule", "when", "time", "timing", "offered", "available"]
            },
            {
                "name": "track_progress",
                "description": "Track student's academic progress and degree requirements. Use when students ask about their progress.",
                "function": self.track_progress,
                "examples": [
                    "How many credits do I have?",
                    "What's my progress toward graduation?",
                    "How many courses do I need to complete?"
                ],
                "keywords": ["progress", "credits", "graduation", "degree", "completed", "remaining"]
            },
            {
                "name": "save_preferences",
                "description": "Save student preferences for learning style, format, or schedule. Use when students express preferences.",
                "function": self.save_preferences,
                "examples": [
                    "I prefer online courses",
                    "Remember that I like hands-on learning",
                    "I want evening classes"
                ],
                "keywords": ["prefer", "like", "remember", "save", "want", "style"]
            }
        ]
    
    def search_courses(self, query: str, limit: int = 5) -> List[Dict]:
        """Search for courses matching the query"""
        courses = self.course_manager.search_courses(query, limit=limit)
        return [{
            "course_code": course.course_code,
            "title": course.title,
            "description": course.description[:100] + "...",
            "level": course.difficulty_level.value,
            "credits": course.credits
        } for course in courses]
    
    def get_recommendations(self, student_profile: Dict, goals: str = "") -> List[Dict]:
        """Get personalized course recommendations"""
        # Simplified recommendation logic
        interests = student_profile.get("interests", [])
        completed = student_profile.get("completed_courses", [])
        
        # Search based on interests
        query = " ".join(interests) + " " + goals
        courses = self.course_manager.search_courses(query, limit=3)
        
        return [{
            "course_code": course.course_code,
            "title": course.title,
            "reason": f"Matches your interest in {', '.join(interests[:2])}",
            "level": course.difficulty_level.value
        } for course in courses]
    
    def check_prerequisites(self, course_code: str, completed_courses: List[str]) -> Dict:
        """Check if prerequisites are met for a course"""
        # Simplified prerequisite checking
        prereq_map = {
            "RU201": ["RU101"],
            "RU202": ["RU101"],
            "RU301": ["RU201"],
            "RU302": ["RU301"]
        }
        
        required = prereq_map.get(course_code, [])
        missing = [req for req in required if req not in completed_courses]
        
        return {
            "course_code": course_code,
            "eligible": len(missing) == 0,
            "required_prerequisites": required,
            "missing_prerequisites": missing
        }
    
    def check_schedule(self, course_code: str = "", semester: str = "") -> Dict:
        """Check course schedule information"""
        # Simplified schedule information
        schedules = {
            "RU101": {"semester": "Fall/Spring", "format": "Online", "duration": "6 weeks"},
            "RU201": {"semester": "Spring", "format": "Online", "duration": "8 weeks"},
            "RU301": {"semester": "Fall", "format": "Hybrid", "duration": "10 weeks"}
        }
        
        if course_code:
            return schedules.get(course_code, {"message": "Schedule information not available"})
        else:
            return {"available_courses": list(schedules.keys()), "schedules": schedules}
    
    def track_progress(self, student_profile: Dict) -> Dict:
        """Track student's academic progress"""
        completed = student_profile.get("completed_courses", [])
        current = student_profile.get("current_courses", [])
        
        # Simplified progress calculation
        total_credits = len(completed) * 3  # Assume 3 credits per course
        required_credits = 30  # Assume 30 credits for specialization
        
        return {
            "completed_courses": len(completed),
            "current_courses": len(current),
            "total_credits": total_credits,
            "required_credits": required_credits,
            "progress_percentage": min(100, (total_credits / required_credits) * 100)
        }
    
    def save_preferences(self, preferences: Dict) -> Dict:
        """Save student preferences"""
        # In a real system, this would save to the memory system
        return {
            "message": "Preferences saved successfully",
            "saved_preferences": preferences
        }

# Initialize the tools
course_manager = CourseManager()
advisor_tools = AcademicAdvisorTools(course_manager)

print(f"Created {len(advisor_tools.tools)} specialized tools:")
for tool in advisor_tools.tools:
    print(f"  - {tool['name']}: {tool['description'][:50]}...")

## Step 3: Build Semantic Tool Selector

Now let's create a semantic tool selector that can intelligently choose the right tool based on user intent.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class SimpleSemanticToolSelector:
    """Semantic tool selector using TF-IDF similarity"""
    
    def __init__(self, tools: List[Dict[str, Any]]):
        self.tools = tools
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
        self._build_tool_index()
    
    def _build_tool_index(self):
        """Build semantic index for tools"""
        # Create searchable text for each tool
        tool_texts = []
        for tool in self.tools:
            # Combine description, examples, and keywords
            text_parts = [
                tool['description'],
                ' '.join(tool['examples']),
                ' '.join(tool['keywords'])
            ]
            tool_texts.append(' '.join(text_parts))
        
        # Create TF-IDF vectors for tools
        self.tool_vectors = self.vectorizer.fit_transform(tool_texts)
        print(f"Built tool index with {self.tool_vectors.shape[1]} features")
    
    def select_tools(self, query: str, max_tools: int = 2, confidence_threshold: float = 0.1) -> List[Tuple[Dict, float]]:
        """Select the most appropriate tools for a query"""
        # Vectorize the query
        query_vector = self.vectorizer.transform([query])
        
        # Calculate similarities with all tools
        similarities = cosine_similarity(query_vector, self.tool_vectors)[0]
        
        # Get tools above confidence threshold
        tool_scores = []
        for i, score in enumerate(similarities):
            if score >= confidence_threshold:
                tool_scores.append((self.tools[i], score))
        
        # Sort by score and return top tools
        tool_scores.sort(key=lambda x: x[1], reverse=True)
        return tool_scores[:max_tools]
    
    def explain_selection(self, query: str, selected_tools: List[Tuple[Dict, float]]) -> str:
        """Explain why tools were selected"""
        if not selected_tools:
            return "No tools matched the query with sufficient confidence."
        
        explanation = f"For query '{query}', selected tools:\n"
        for tool, score in selected_tools:
            explanation += f"  - {tool['name']} (confidence: {score:.3f}): {tool['description'][:60]}...\n"
        
        return explanation

# Initialize the semantic tool selector
tool_selector = SimpleSemanticToolSelector(advisor_tools.tools)

# Test tool selection with different queries
test_queries = [
    "What machine learning courses are available?",
    "What should I take next semester?",
    "Can I enroll in RU301?",
    "I prefer online classes",
    "How many credits do I have?"
]

print("\nTesting semantic tool selection:")
print("=" * 50)

for query in test_queries:
    selected_tools = tool_selector.select_tools(query, max_tools=2)
    print(f"\nQuery: '{query}'")
    
    if selected_tools:
        for tool, score in selected_tools:
            print(f"  → {tool['name']} (confidence: {score:.3f})")
    else:
        print("  → No tools selected")

print("\nSemantic tool selection working!")

## Step 4: Build Multi-Tool Agent

Let's create an enhanced agent that combines memory with intelligent tool selection.

In [ ]:
class MultiToolAgent:
    """Enhanced agent with memory and semantic tool selection"""
    
    def __init__(self, advisor_tools: AcademicAdvisorTools, tool_selector: SimpleSemanticToolSelector):
        self.advisor_tools = advisor_tools
        self.tool_selector = tool_selector
        
        # Memory system (simplified from Section 3)
        self.working_memory = {
            "conversation_history": [],
            "tool_usage_history": [],
            "session_context": {}
        }
        self.long_term_memory = {}  # Keyed by student email
        
        self.current_student = None
        self.session_id = None
    
    def start_session(self, student: StudentProfile) -> str:
        """Start a new session with memory loading"""
        self.session_id = f"{student.email}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.current_student = student
        
        # Clear working memory
        self.working_memory = {
            "conversation_history": [],
            "tool_usage_history": [],
            "session_context": {
                "student_profile": {
                    "name": student.name,
                    "email": student.email,
                    "major": student.major,
                    "year": student.year,
                    "completed_courses": student.completed_courses,
                    "interests": student.interests,
                    "preferred_format": student.preferred_format.value,
                    "preferred_difficulty": student.preferred_difficulty.value
                }
            }
        }
        
        # Load long-term memory
        if student.email in self.long_term_memory:
            self.working_memory["loaded_memories"] = self.long_term_memory[student.email]
            print(f"Loaded {len(self.long_term_memory[student.email])} memories for {student.name}")
        else:
            self.working_memory["loaded_memories"] = []
            print(f"Starting fresh session for {student.name}")
        
        return self.session_id
    
    def _enhance_query_with_memory(self, query: str) -> str:
        """Enhance query with relevant memory context for better tool selection"""
        enhanced_query = query
        
        # Add student interests to query context
        if self.current_student:
            interests = " ".join(self.current_student.interests)
            enhanced_query += f" student interests: {interests}"
        
        # Add recent conversation context
        recent_messages = self.working_memory["conversation_history"][-2:]
        for msg in recent_messages:
            if msg["role"] == "user":
                enhanced_query += f" previous: {msg['content']}"
        
        return enhanced_query
    
    def _execute_tool(self, tool: Dict[str, Any], query: str) -> Dict[str, Any]:
        """Execute a selected tool with appropriate parameters"""
        tool_name = tool["name"]
        tool_function = tool["function"]
        
        try:
            # Prepare parameters based on tool type
            if tool_name == "search_courses":
                result = tool_function(query)
            
            elif tool_name == "get_recommendations":
                student_profile = self.working_memory["session_context"]["student_profile"]
                result = tool_function(student_profile, query)
            
            elif tool_name == "check_prerequisites":
                # Extract course code from query (simplified)
                course_code = "RU301"  # Would need better extraction in real system
                completed = self.working_memory["session_context"]["student_profile"]["completed_courses"]
                result = tool_function(course_code, completed)
            
            elif tool_name == "check_schedule":
                result = tool_function()
            
            elif tool_name == "track_progress":
                student_profile = self.working_memory["session_context"]["student_profile"]
                result = tool_function(student_profile)
            
            elif tool_name == "save_preferences":
                # Extract preferences from query (simplified)
                preferences = {"query": query}
                result = tool_function(preferences)
            
            else:
                result = {"error": f"Unknown tool: {tool_name}"}
            
            # Log tool usage
            self.working_memory["tool_usage_history"].append({
                "tool_name": tool_name,
                "query": query,
                "result": result,
                "timestamp": datetime.now().isoformat()
            })
            
            return result
            
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}"}
    
    def chat(self, query: str) -> str:
        """Main chat method with tool selection and execution"""
        if not self.current_student:
            return "Please start a session first."
        
        # Add to conversation history
        self.working_memory["conversation_history"].append({
            "role": "user",
            "content": query,
            "timestamp": datetime.now().isoformat()
        })
        
        # Enhance query with memory context
        enhanced_query = self._enhance_query_with_memory(query)
        
        # Select appropriate tools
        selected_tools = self.tool_selector.select_tools(enhanced_query, max_tools=2)
        
        if not selected_tools:
            response = "I'm not sure how to help with that. Could you rephrase your question?"
        else:
            # Execute the best tool
            best_tool, confidence = selected_tools[0]
            tool_result = self._execute_tool(best_tool, query)
            
            # Generate response based on tool result
            response = self._generate_response(best_tool, tool_result, query)
        
        # Add response to conversation history
        self.working_memory["conversation_history"].append({
            "role": "assistant",
            "content": response,
            "timestamp": datetime.now().isoformat()
        })
        
        return response
    
    def _generate_response(self, tool: Dict[str, Any], tool_result: Dict[str, Any], query: str) -> str:
        """Generate natural language response from tool result"""
        tool_name = tool["name"]
        
        if "error" in tool_result:
            return f"I encountered an error: {tool_result['error']}"
        
        if tool_name == "search_courses":
            courses = tool_result
            if courses:
                response = f"I found {len(courses)} courses for you:\n"
                for course in courses[:3]:
                    response += f"• {course['course_code']}: {course['title']} ({course['level']} level)\n"
                return response
            else:
                return "I couldn't find any courses matching your criteria."
        
        elif tool_name == "get_recommendations":
            recommendations = tool_result
            if recommendations:
                response = "Based on your profile, I recommend:\n"
                for rec in recommendations:
                    response += f"• {rec['course_code']}: {rec['title']} - {rec['reason']}\n"
                return response
            else:
                return "I couldn't generate specific recommendations right now."
        
        elif tool_name == "track_progress":
            progress = tool_result
            return f"Your academic progress: {progress['completed_courses']} courses completed, {progress['total_credits']} credits earned. You're {progress['progress_percentage']:.1f}% toward your goal."
        
        else:
            return f"I used the {tool_name} tool and got: {str(tool_result)}"

# Initialize the multi-tool agent
multi_tool_agent = MultiToolAgent(advisor_tools, tool_selector)

print("Multi-tool agent initialized with memory and semantic tool selection")

## Step 5: Test Multi-Tool Scenarios

Let's test the multi-tool agent with complex scenarios that require different tools.

In [ ]:
# Create test student
alex = StudentProfile(
    name="Alex Rodriguez",
    email="alex.r@university.edu",
    major="Data Science",
    year=2,
    completed_courses=["RU101"],
    current_courses=[],
    interests=["machine learning", "python", "data analysis"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE,
    max_credits_per_semester=12
)

# Start session
session_id = multi_tool_agent.start_session(alex)

print("TESTING MULTI-TOOL SCENARIOS")
print("=" * 50)

# Test different types of queries
test_scenarios = [
    {
        "query": "What machine learning courses are available?",
        "expected_tool": "search_courses",
        "description": "Course discovery query"
    },
    {
        "query": "What should I take next based on my background?",
        "expected_tool": "get_recommendations",
        "description": "Personalized recommendation query"
    },
    {
        "query": "How many credits do I have so far?",
        "expected_tool": "track_progress",
        "description": "Progress tracking query"
    },
    {
        "query": "I prefer online courses with hands-on projects",
        "expected_tool": "save_preferences",
        "description": "Preference saving query"
    },
    {
        "query": "Can I take the advanced vector search course?",
        "expected_tool": "check_prerequisites",
        "description": "Prerequisite checking query"
    }
]

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\nScenario {i}: {scenario['description']}")
    print(f"Query: '{scenario['query']}'")
    
    # Get tool selection first
    selected_tools = tool_selector.select_tools(scenario['query'], max_tools=1)
    if selected_tools:
        selected_tool_name = selected_tools[0][0]['name']
        confidence = selected_tools[0][1]
        print(f"Selected tool: {selected_tool_name} (confidence: {confidence:.3f})")
    
    # Get agent response
    response = multi_tool_agent.chat(scenario['query'])
    print(f"Agent response: {response[:100]}...")
    print("-" * 30)

print("\nMulti-tool scenarios completed successfully!")

## Step 6: Test Memory-Aware Tool Selection

Let's test how memory context improves tool selection accuracy.

In [ ]:
print("TESTING MEMORY-AWARE TOOL SELECTION")
print("=" * 50)

# Create a conversation sequence to build context
conversation_sequence = [
    "I'm interested in machine learning for my thesis research",
    "What courses would help me with ML applications?",
    "That sounds good. Can I take that course?",  # Reference to previous recommendation
    "How much progress would that give me toward graduation?"
]

print("Building conversation context...\n")

for i, query in enumerate(conversation_sequence, 1):
    print(f"Turn {i}: {query}")
    
    # Show tool selection without memory enhancement
    basic_tools = tool_selector.select_tools(query, max_tools=1)
    basic_tool_name = basic_tools[0][0]['name'] if basic_tools else "none"
    
    # Show tool selection with memory enhancement
    enhanced_query = multi_tool_agent._enhance_query_with_memory(query)
    enhanced_tools = tool_selector.select_tools(enhanced_query, max_tools=1)
    enhanced_tool_name = enhanced_tools[0][0]['name'] if enhanced_tools else "none"
    
    print(f"  Basic selection: {basic_tool_name}")
    print(f"  Memory-enhanced: {enhanced_tool_name}")
    
    # Get actual response (builds conversation history)
    response = multi_tool_agent.chat(query)
    print(f"  Response: {response[:80]}...")
    print()

print("Memory-aware tool selection demonstration complete!")

# Show conversation history
print("\nConversation History:")
for msg in multi_tool_agent.working_memory["conversation_history"][-4:]:
    role = msg["role"].title()
    content = msg["content"][:60] + "..." if len(msg["content"]) > 60 else msg["content"]
    print(f"  {role}: {content}")

# Show tool usage history
print("\nTool Usage History:")
for usage in multi_tool_agent.working_memory["tool_usage_history"][-3:]:
    print(f"  {usage['tool_name']}: {usage['query'][:40]}...")

## Step 7: Tool Selection Analysis

Let's analyze how the semantic tool selection system works and its effectiveness.

In [ ]:
# Analyze tool selection patterns
print("TOOL SELECTION ANALYSIS")
print("=" * 40)

# Test edge cases and ambiguous queries
edge_case_queries = [
    "Help me with courses",  # Ambiguous
    "I need information",    # Very vague
    "What about RU301?",     # Context-dependent
    "Show me everything",    # Overly broad
    "Can you help?",         # Generic
]

print("\nEdge Case Analysis:")
print("-" * 30)

for query in edge_case_queries:
    selected_tools = tool_selector.select_tools(query, max_tools=2, confidence_threshold=0.05)
    print(f"\nQuery: '{query}'")
    
    if selected_tools:
        for tool, confidence in selected_tools:
            print(f"  → {tool['name']} (confidence: {confidence:.3f})")
    else:
        print(f"  → No tools selected (all below threshold)")

# Analyze tool coverage
print("\n\nTool Coverage Analysis:")
print("-" * 30)

tool_usage_count = {}
test_queries_comprehensive = [
    "Find machine learning courses",
    "What should I study next?",
    "Check my academic progress",
    "I prefer online learning",
    "Can I take advanced courses?",
    "When are courses offered?",
    "Show available courses",
    "Recommend courses for data science",
    "How many credits do I need?",
    "Remember my learning preferences"
]

for query in test_queries_comprehensive:
    selected_tools = tool_selector.select_tools(query, max_tools=1)
    if selected_tools:
        tool_name = selected_tools[0][0]['name']
        tool_usage_count[tool_name] = tool_usage_count.get(tool_name, 0) + 1

print("Tool usage distribution:")
for tool_name, count in sorted(tool_usage_count.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tool_name}: {count} queries")

# Calculate coverage
total_tools = len(advisor_tools.tools)
used_tools = len(tool_usage_count)
coverage = (used_tools / total_tools) * 100

print(f"\nTool coverage: {used_tools}/{total_tools} tools used ({coverage:.1f}%)")

# Show unused tools
all_tool_names = {tool['name'] for tool in advisor_tools.tools}
used_tool_names = set(tool_usage_count.keys())
unused_tools = all_tool_names - used_tool_names

if unused_tools:
    print(f"Unused tools: {', '.join(unused_tools)}")
    print("Consider improving descriptions or adding more diverse test queries.")
else:
    print("All tools are being selected by the test queries.")

print("\nTool selection analysis complete!")

## Step 8: Multi-Tool Architecture Summary

Let's review what you've built and how it prepares you for the final section.

In [ ]:
# Multi-tool architecture summary
print("MULTI-TOOL ARCHITECTURE SUMMARY")
print("=" * 50)

architecture_components = {
    "Specialized Tools": {
        "count": len(advisor_tools.tools),
        "purpose": "Domain-specific capabilities for academic advising",
        "examples": ["search_courses", "get_recommendations", "check_prerequisites"],
        "next_enhancement": "Section 5: Tool performance optimization"
    },
    "Semantic Tool Selector": {
        "count": 1,
        "purpose": "Intelligent tool routing based on query intent",
        "examples": ["TF-IDF similarity", "Confidence scoring", "Multi-tool selection"],
        "next_enhancement": "Section 5: Embedding-based selection"
    },
    "Memory Integration": {
        "count": 1,
        "purpose": "Memory-aware tool selection and execution",
        "examples": ["Query enhancement", "Context loading", "Tool usage history"],
        "next_enhancement": "Section 5: Memory-optimized routing"
    },
    "Multi-Tool Agent": {
        "count": 1,
        "purpose": "Orchestrates tool selection, execution, and response generation",
        "examples": ["Session management", "Tool execution", "Response synthesis"],
        "next_enhancement": "Section 5: Production scaling and optimization"
    }
}

for component, details in architecture_components.items():
    print(f"\n{component}:")
    print(f"  Purpose: {details['purpose']}")
    print(f"  Count: {details['count']}")
    print(f"  Examples: {', '.join(details['examples'])}")
    print(f"  Next enhancement: {details['next_enhancement']}")

print("\nKey Improvements Over Section 3:")
improvements = [
    "Multiple specialized tools instead of single RAG pipeline",
    "Semantic tool selection with confidence scoring",
    "Memory-aware query enhancement for better tool routing",
    "Tool usage tracking and analysis",
    "Complex multi-turn conversation support",
    "Intent classification and tool orchestration"
]

for improvement in improvements:
    print(f"  - {improvement}")

print("\nAgent Evolution Summary:")
evolution_stages = {
    "Section 2": "Basic RAG agent with simple course search",
    "Section 3": "Memory-enhanced agent with conversation persistence",
    "Section 4": "Multi-tool agent with semantic routing and specialized capabilities",
    "Section 5": "Production-optimized agent with efficiency and scaling"
}

for section, description in evolution_stages.items():
    status = "✅ Complete" if section != "Section 5" else "🔄 Next"
    print(f"  {section}: {description} {status}")

print("\nReady for Section 5: Context Optimization!")
print("Your multi-tool agent now has the foundation for production-grade optimization.")

## Key Takeaways

Congratulations! You've successfully built a sophisticated multi-tool agent with semantic tool selection. Here's what you accomplished:

### What You Built
1. **Specialized Tool Suite** - Six domain-specific tools for comprehensive academic advising
2. **Semantic Tool Selector** - Intelligent routing based on query intent and similarity
3. **Memory-Aware Tool Selection** - Enhanced query context using conversation and user memory
4. **Multi-Tool Agent** - Orchestrates tool selection, execution, and response generation
5. **Tool Usage Analytics** - Tracking and analysis of tool selection patterns

### Key Tool Selection Concepts Mastered
- **Intent Classification**: Understanding what users want to accomplish
- **Semantic Similarity**: Matching queries to tool capabilities using vector similarity
- **Confidence Scoring**: Measuring certainty in tool selection decisions
- **Memory Integration**: Using conversation context to improve tool routing
- **Tool Orchestration**: Managing multiple tools in a cohesive system

### Cross-Reference with Original Notebooks
This implementation builds on concepts from:
- `section-2-system-context/02_defining_tools.ipynb` - Tool definition and schema design
- `section-2-system-context/03_tool_selection_strategies.ipynb` - Tool selection challenges and strategies
- Reference-agent's `semantic_tool_selector.py` - Production-ready semantic routing patterns

### Production-Ready Patterns
- **Modular Tool Architecture** - Easy to add, remove, or modify individual tools
- **Confidence-Based Selection** - Handles ambiguous queries gracefully
- **Memory-Enhanced Routing** - Leverages conversation context for better decisions
- **Tool Usage Analytics** - Monitoring and optimization capabilities
- **Error Handling** - Graceful degradation when tools fail

### Agent Capabilities Now Include
- **Course Discovery**: "What machine learning courses are available?"
- **Personalized Recommendations**: "What should I take next based on my background?"
- **Prerequisite Checking**: "Can I take the advanced vector search course?"
- **Progress Tracking**: "How many credits do I have so far?"
- **Schedule Information**: "When are courses offered this semester?"
- **Preference Management**: "I prefer online courses with hands-on projects"

### What's Next
Your multi-tool agent is now ready for production optimization:
- **Context Optimization** - Efficient memory usage and token management
- **Performance Scaling** - Handle thousands of concurrent users
- **Cost Optimization** - Minimize API calls and computational overhead
- **Advanced Analytics** - Sophisticated monitoring and improvement strategies

The sophisticated tool selection architecture you've built provides the foundation for production-grade context engineering systems.

---

**Continue to Section 5: Context Optimization**